# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

## Open the data file and have a look at the general information. 

In [1]:
#Open and read datafile
import pandas as pd
df = pd.read_csv('/datasets/credit_scoring_eng.csv')

#Look at general info
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


# Conclusion

The data file consists of 21525 entries and a total of 12 columns. There are missing quantitative values within the 'days_employed' and 'total_income' columns. Both columns contain values with the float64 datatype. The 'days_employed' column also contains negative values. These negative values most likely represent the amount of days that a customer has been unemployed. 

## Data preprocessing

### Processing missing values

In [2]:
#Finding total missing values
print("Total income missing values count:",df['total_income'].isnull().sum())
print("Days employed missing values count:",df['days_employed'].isnull().sum())
print()

#Finding median and mean
median_income = df['total_income'].median()
avg_income = df['total_income'].mean()

print("Average income:",avg_income)
print("Median income:",median_income)
print()

avg_days_employed = df['days_employed'].mean()
median_days = df['days_employed'].median()
print("Average days employed:",avg_days_employed)
print("Median days employed:",median_days)

#Filling in values
df['total_income'].fillna(median_income,inplace=True)
df['days_employed'].fillna(avg_income,inplace=True)
print()

#Conclusion
print("Total income column missing values count:",df['total_income'].isna().sum())
print("Days employed column missing values count:", df['days_employed'].isna().sum())
print()

df.info()



Total income missing values count: 2174
Days employed missing values count: 2174

Average income: 26787.56835465867
Median income: 23202.87

Average days employed: 63046.49766147338
Median days employed: -1203.369528770489

Total income column missing values count: 0
Days employed column missing values count: 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Conclusion

There are 2174 missing values each in both the 'total_income' and 'days_employed' columns. The fillna() method was used on each column. For the 'total_income' column, either the mean or the median could've been used to fill in the missing values. For the 'days_employed' column the mean was used to fill in the values because there are significant outliers that may skew our data. Possible reasons for the missing values may be human errors while filling out a loan application such as blank values, etc. 

### Data type replacement

In [3]:
df.info()

#Changing data types to integer
try:
    df['days_employed'].to_numeric()
except:
    df['days_employed'] = df['days_employed'].astype('int')
    print()
    print('days_employed column changed to "int" datatype')
try:
    df['days_employed'].to_numeric()
except:
    df['total_income'] = df['total_income'].astype('int')
    print()
    print('total_income column changed to "int" datatype')

print()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB

days_employed column changed to "int" datatype

total_income column changed to "int" datatype

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64


### Conclusion

The data types were float-64. Used a try-except clause with the to_numeric() and astype() methods on both columns. The to_numeric() code crashed because both columns contained no string values. Using ('int') as the astype() method's argument, the column data types were converted to the integer type. The astype() method was used in the except because the columns contained quantitative values as opposed to categorical values.

### Processing duplicates

In [4]:
#Finding total number of duplicates in entire dataframe
print("Total duplicates:",df.duplicated().sum())

#Deleting duplicates
df = df.drop_duplicates()

print("Total duplicates:",df.duplicated().sum())







Total duplicates: 54
Total duplicates: 0


### Conclusion

The duplicated() method was used together with sum() to get the number of duplicate values in the entire dataframe rather than each column. The dataframe was found to have 54 duplicates, which were subsequently dropped using the drop_duplicates() method. A possible reason for the appearance of duplicates is that customers may have applied several times for a loan with the same information.


### Categorizing Data

In [5]:
#Creating function to categorize income level
def income_level(income):
    if income <= 50000:
        return 'low income'
    if income <= 150000:
        return 'middle class'
    if income > 150000:
        return 'high income'

#Adding new column 'income_level'
df['income_level'] = df['total_income'].apply(income_level)

#Creating function to categorize customers with children
def has_children(number):
    if number <= 0:
        return "No"
    elif number > 0:
        return "Yes"

df['has_children'] = df['children'].apply(has_children)







# Conclusion

To find the relevant data, the data needs to be categorized and organized  into groups according to specific criteria in order to draw meaningful conclusions. The income_level and has_children were created and added to the dataframe using the apply() method.  

## Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [6]:
#categorizing data to find correlation between children and debt


print(df['has_children'].value_counts())
print()


print(pd.pivot_table(df,columns='has_children',values='debt',aggfunc='sum'))


No     14154
Yes     7317
Name: has_children, dtype: int64

has_children    No  Yes
debt          1064  677


According to the description of the data given, the debt column refers to whether the customer has ever defaulted on a loan. The 'debt' column only contains 0 and 1 values corresponding to 'no' and 'yes' responses. Creating a pivot table with the aggfunc parameter set to 'sum', we can aggregate the total 'yes' responses with the 'has_children' columns to get the relevant data. Looking at the data, there is no significant relation between having children and defaulting a loan.  

### Conclusion

Customers with children significantly outnumber customers without children in paying loans on time.

- Is there a relation between marital status and repaying a loan on time?

In [7]:
#categorizing data to find correlation between marital_status and debt

print(df['family_status'].value_counts())

print()

print(pd.pivot_table(df,index='family_status',values='debt',aggfunc='sum'))




married              12344
civil partnership     4163
unmarried             2810
divorced              1195
widow / widower        959
Name: family_status, dtype: int64

                   debt
family_status          
civil partnership   388
divorced             85
married             931
unmarried           274
widow / widower      63


According to the pivot table, married customers defaulted on a loan the most out of each marital status, followed by customers in a civil partnership. Using the value_counts() method on the family_status column, we can see that married customers are the majority of the applicants for a loan.

### Conclusion

- Is there a relation between income level and repaying a loan on time?

In [8]:
#categorizing data to find correlation between income level and debt

print(df['income_level'].value_counts())
print()
print(pd.pivot_table(df,columns='income_level',values='debt',aggfunc='sum'))



low income      20151
middle class     1292
high income        28
Name: income_level, dtype: int64

income_level  high income  low income  middle class
debt                    2        1649            90


According to the pivot table, we can see a correlation between income level and repaying a loan on time. The majority of applicants were low income (20k) and customers that defaulted were of a lower income level.  

### Conclusion

- How do different loan purposes affect on-time repayment of the loan?

In [9]:
print(df.groupby('purpose')['debt'].sum().sort_values(ascending=False))

import pandas as pd
from nltk.stem import SnowballStemmer
english_stemmer = SnowballStemmer('english')

def categorize(purpose):
    for word in purpose.split(' '):
        stem = english_stemmer.stem(word)
        if stem == 'hous' or stem == 'estat' or stem == 'properti':
            return 'housing'
        if stem == 'car':
            return 'car purchase'
        if stem == 'educ' or stem == 'univers':
            return 'education'
        if stem == 'wed':
            return 'wedding'
    
    return 'unknown'


df['purpose_category'] = df['purpose'].apply(categorize)

df.pivot_table(index='purpose_category',values='debt',aggfunc='sum')




purpose
wedding ceremony                            64
having a wedding                            64
to have a wedding                           58
real estate transactions                    55
building a property                         54
buying property for renting out             52
transactions with commercial real estate    52
to get a supplementary education            51
second-hand car purchase                    51
transactions with my real estate            50
purchase of a car                           50
building a real estate                      48
housing transactions                        48
to own a car                                48
purchase of the house                       48
buy commercial real estate                  47
buying my own car                           46
housing                                     46
getting higher education                    46
purchase of the house for my family         45
cars                                        44
profi

,debt
purpose_category,
car purchase,403
education,370
housing,782
wedding,186


### Conclusion

The 'purpose' column was divided into categories by stemmming. The loan purposes with the most loan defaults were housing related.

## General conclusion

The data file consists of 21525 entries and a total of 12 columns. There are missing quantitative values within the 'days_employed' and 'total_income' columns, with each column's datatype being float-64. Possible reasons for the missing values may be human errors while filling out a loan application such as blank values, etc. The missing values were filled in using the fillna() method. The column's datatypes were changed to int using the astype() method. The dataframe was found to have 54 duplicates, which were subsequently dropped using the drop_duplicates() method. A possible reason for the appearance of duplicates is that customers may have applied several times for a loan with the same information.

To find the relevant data, the data needs to be categorized and organized  into groups according to specific criteria in order to draw meaningful conclusions. The income_level and has_children were created and added to the dataframe using the apply() method.  According to the categorized data, there is no significant relation between having children and defaulting a loan. Married customers defaulted on a loan the most out of each marital status, followed by customers in a civil partnership. The majority of applicants were low income and customers that defaulted were of a lower income level. The loan purposes with the most loan defaults were car related.
